In [8]:
!pip  install transformers
!pip install pytorch-ignite

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
import numpy as np
import pandas as pd 
import os
import gc
import copy
import time
import random
import string
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import defaultdict
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

In [2]:
from transformers import AutoTokenizer, AutoModel, AdamW

In [3]:
!pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from urllib import request

In [5]:
df=pd.read_csv('train.En.csv')
df=df[['tweet','sarcastic']]

In [6]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [7]:
train=pd.concat([train, validate], ignore_index=True)

In [9]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

2024-04-22 15:29:03.100999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 15:29:03.215316: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [10]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [11]:
class PCLTrainDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['tweet'].values
        self.label = df['sarcastic'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )         
        target = self.label[index]
        
        text_ids = inputs_text['input_ids']
        text_mask = inputs_text['attention_mask']
        
        return {
            'text_ids': torch.tensor(text_ids, dtype=torch.long),
            'text_mask': torch.tensor(text_mask, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.float)
        }

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class F1_Loss(nn.Module):
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true):
        y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2).to(torch.float32)
        
        tp = (y_true_one_hot * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true_one_hot) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true_one_hot) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true_one_hot * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)

        CE = torch.nn.CrossEntropyLoss(weight=(1 - f1))(y_pred, y_true_one_hot)
        return CE.mean()
    
def criterion(outputs1,  targets):
    criterion = F1_Loss()
    loss = criterion(outputs1, targets)
    return loss

def sigmoid(x):
    return 1/(1+math.exp(-x))

In [13]:
class PCL_Model_Arch(nn.Module):
    def __init__(self,pre_trained='vinai/bertweet-base'):
        super().__init__()
        
        self.bert = AutoModel.from_pretrained(pre_trained)
        self.hidden_size = self.bert.config.hidden_size
        self.LSTM = nn.LSTM(self.hidden_size,self.hidden_size,bidirectional=True)
        self.clf = nn.Linear(self.hidden_size*2,2)
        
    def forward(self,id,mask):
        
        encoded_layers = self.bert(input_ids=id,attention_mask=mask)
        encoded_layers = encoded_layers[0].permute(1, 0, 2)
        enc_hiddens, (last_hidden, last_cell) = self.LSTM(encoded_layers)
        output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)
        output_hidden = F.dropout(output_hidden,0.2)
        output = self.clf(output_hidden)
        
        return F.softmax(output)

In [14]:
tokenizer= AutoTokenizer.from_pretrained('vinai/bertweet-base')

In [15]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        
        text_ids = data['text_ids'].to(device, dtype=torch.long)
        text_mask = data['text_mask'].to(device, dtype=torch.long)
        targets = data['target'].to(device, dtype=torch.long)       
        batch_size = text_ids.size(0)
        outputs = model(text_ids, text_mask)       
        loss = criterion(outputs, targets)
        loss = loss / 1
        loss.backward()
    
        if (step + 1) % 1 == 0:
            optimizer.step()
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [16]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        
        text_ids = data['text_ids'].to(device, dtype=torch.long)
        text_mask = data['text_mask'].to(device, dtype=torch.long)
        targets = data['target'].to(device, dtype=torch.long)       
        batch_size = text_ids.size(0)
        outputs = model(text_ids, text_mask)       
        loss = criterion(outputs, targets)
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss

In [22]:
def run_training(model, optimizer, scheduler, device, num_epochs):
    # To automatically log gradients   
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=device, epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=device, 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
       
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"bert_tweet/Loss-Fold-0.bin"
            torch.save(model.state_dict(), PATH)
            print("Model Saved")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [23]:
def prepare_loaders(train_df, valid_df):
    train_dataset = PCLTrainDataset(train_df, tokenizer=tokenizer, max_length=120)
    valid_dataset = PCLTrainDataset(valid_df, tokenizer=tokenizer, max_length=120)

    train_loader = DataLoader(train_dataset, batch_size=16, 
                              num_workers=2, shuffle=False, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset, batch_size=16, 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader


In [24]:
import gc
gc.collect()

5807

In [25]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(train, test_size=0.1, random_state=42)

train_loader, valid_loader = prepare_loaders(train_df, valid_df)

model = PCL_Model_Arch()
model.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
torch.cuda.empty_cache()

optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-6)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=500, eta_min=1e-6)

model, history = run_training(model, optimizer, scheduler,
                              device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
                              num_epochs=3)

del model, history, train_loader, valid_loader
_ = gc.collect()
print()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 29.06it/s, Epoch=1, LR=7.81e-5, Valid_Loss=0.243]


Validation Loss Improved (inf ---> 0.24320906121953786)
Model Saved



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 29.23it/s, Epoch=2, LR=3.17e-5, Valid_Loss=0.242]


Validation Loss Improved (0.24320906121953786 ---> 0.24242015708264686)
Model Saved



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 29.40it/s, Epoch=3, LR=2e-6, Valid_Loss=0.241]


Validation Loss Improved (0.24242015708264686 ---> 0.24116663281008494)
Model Saved

Training complete in 0h 0m 48s
Best Loss: 0.2412



In [26]:
train_loader, valid_loader = prepare_loaders(train_df, valid_df)

model = PCL_Model_Arch()
model.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
torch.cuda.empty_cache()

# Define Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=1e-6)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=500, eta_min=1e-6)

# Train the model
model, history = run_training(model, optimizer, scheduler,
                              device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
                              num_epochs=3)

# Clean up
del model, history, train_loader, valid_loader
_ = gc.collect()
print()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 28.86it/s, Epoch=1, LR=7.81e-5, Valid_Loss=0.244]


Validation Loss Improved (inf ---> 0.24422530100928794)
Model Saved



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 29.22it/s, Epoch=2, LR=3.17e-5, Valid_Loss=0.244]


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 29.22it/s, Epoch=3, LR=2e-6, Valid_Loss=0.24]


Validation Loss Improved (0.24422530100928794 ---> 0.23996330379582137)
Model Saved

Training complete in 0h 0m 47s
Best Loss: 0.2400



In [27]:
test.dropna(inplace=True)

In [28]:
valid_dataset = PCLTrainDataset(test, tokenizer=tokenizer, max_length=120)
valid_loader = DataLoader(valid_dataset, batch_size=16, 
                         num_workers=2, shuffle=False, pin_memory=True)

In [29]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['text_ids'].to(device, dtype = torch.long)
        mask = data['text_mask'].to(device, dtype = torch.long)
        outputs = model(ids, mask)
        PREDS.append(outputs.detach().cpu().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [30]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = PCL_Model_Arch()
        model.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    final_preds= np.argmax(final_preds,axis=1)
    return final_preds

In [31]:
#MODEL_PATH_2=['bert_tweet/Loss-Fold-0.bin','bert_tweet/Loss-Fold-1.bin','bert_tweet/Loss-Fold-2.bin','bert_tweet/Loss-Fold-3.bin','bert_tweet/Loss-Fold-4.bin']
MODEL_PATH_2=['bert_tweet/Loss-Fold-0.bin']
preds = inference(MODEL_PATH_2, valid_loader, torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

Getting predictions for model 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:01<00:00, 32.74it/s]


In [32]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

In [33]:
print(print_statistics(test['sarcastic'],preds))

Accuracy: 0.766
Precision: 0.587
Recall: 0.766
F_score: 0.665

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       531
           1       0.00      0.00      0.00       162

    accuracy                           0.77       693
   macro avg       0.38      0.50      0.43       693
weighted avg       0.59      0.77      0.66       693

(0.7662337662337663, 0.587114184516782, 0.7662337662337663, 0.664820473644003)
